In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import csv
import json
from datetime import datetime
path = os.getcwd()
path
pd.set_option('display.max_rows', None)

In [2]:
class parse:
    def __init__(self,data_root):
        self.data_root = data_root
        self.dfs = self.get_dfs(self.data_root)
        
    def get_dfs(self,data_root):
        dfs = {'df1':'','df2':'','df3':'','df4':'','df5':'','df6':'','df7':'','df8':''}
        
        csv_files = glob.glob('{}/*'.format(data_root))
        count = 0
        for csv_file in csv_files:
            count += 1
            print(csv_file)
        #     print(csv_file.split('/')[-1])
            pre_data_X = pd.read_csv(csv_file, header=None,sep='/s',engine='python')
        #     print(pre_data_X)
            pre_data_X[2] = pre_data_X[0] + pre_data_X[1]
            df_name = 'df'+str(count)
            dfs[df_name]= self.build_df(pre_data_X)
        return dfs
    
    def build_df(self,pre_data_df):
        column_names = []
        rows_data = []
        for data in pre_data_df[2]:
            tmp = []
            dict_data = json.loads(data)
            self.recursive_key(None,dict_data,tmp)
            if not column_names:

                column_names = ['']*len(tmp)
            for i in range(len(tmp)):
                column_names[i] = tmp[i][0:tmp[i].index(':')]
    #         print(tmp)
    #         print(len(column_names))
            row_data = ['']*len(column_names)
            for i in range(len(tmp)):
                row_data[i] = tmp[i][tmp[i].index(':')+1:]
            rows_data.append(row_data)
    #     print(rows_data)
        df = pd.DataFrame(rows_data,columns=column_names)
        df['life'] = df['data.context.timeStamp'].apply\
                        (lambda x:self.cal_left_life('2018-07-08T22:08:15+00:00',x))
        return df
    
    def recursive_key(self,root,dic,l):
        if isinstance(dic,list):
            if 'set' in root:
                l.append(str(root)+":"+str(dic))
                return
            else:
                for item in dic:
                    next_root = root+'.'+item['name']
                    l.append(str(next_root)+':'+str(item['value']))
        elif isinstance(dic,dict):
            for k,v in dic.items():
                if not root:
    #                 l.append(v)
                    next_root = str(k)
                else:
                    next_root = root+'.'+k

                self.recursive_key(next_root,v,l)
        else:
            l.append(str(root)+':'+str(dic))
    
    def cal_left_life(self,end,cur):
        def parse(time):
            time = time.split('+')[0]
            date = time.split('T')[0]
            day_time = time.split('T')[1]
            return date + ' ' + day_time
        end_time = datetime.strptime(parse(end),"%Y-%m-%d %H:%M:%S")
        cur_time = datetime.strptime(parse(cur),"%Y-%m-%d %H:%M:%S")
        life = (end_time-cur_time).days
        return life

In [3]:
#修改文件夹路经
data_root = '/home/alexhang/project/iiac/nengyuan/data/train/67'
dfs_parse = parse(data_root)
dfs = dfs_parse.dfs
dfs.keys()

/home/alexhang/project/iiac/nengyuan/data/train/67/Env_Time_200_Tr.csv
/home/alexhang/project/iiac/nengyuan/data/train/67/Env_200_Tr.csv
/home/alexhang/project/iiac/nengyuan/data/train/67/Time_62_Tr.csv
/home/alexhang/project/iiac/nengyuan/data/train/67/FFT_0-1000_Tr.csv
/home/alexhang/project/iiac/nengyuan/data/train/67/FFT_0-1000.csv
/home/alexhang/project/iiac/nengyuan/data/train/67/Time_1000.csv
/home/alexhang/project/iiac/nengyuan/data/train/67/Time_1000_Tr.csv
/home/alexhang/project/iiac/nengyuan/data/train/67/FFT_62_Tr.csv


dict_keys(['df1', 'df2', 'df3', 'df4', 'df5', 'df6', 'df7', 'df8'])

In [4]:
dfs['df1'].head(5)

,details.name,details.type,details.sensorName,data.context.timeStamp,data.context.rpm,data.context.duration,data.context.binningParameters.WPS-ActivePower-Average,data.context.binningParameters.WPS-ActivePower-Minimum,data.context.binningParameters.WPS-ActivePower-Maximum,data.context.binningParameters.WPS-ActivePower-Deviation,...,data.context.operationalValues.WPS-Gearoiltemperature,data.context.operationalValues.WPS-GeneratorRPM,data.context.operationalValues.WPS-PitchReference,data.context.operationalValues.WPS-RotorRPM,data.context.operationalValues.WPS-Windspeed,data.context.operationalValues.WPS-YawAngle,data.context.operationalValues.bias voltage,data.context.condition,data.set,life
0,Env_Time_200_Tr,TrackedEnvelopeTimeSeries,Main bearing,2018-07-08T22:08:15+00:00,1500,157,0.20424615,0.18884615,0.23,0.03052885,...,0.76666667,991.58,-0.01858444,10.942,0.238624,0.99022222,12.3,WPS-ActivePower 0.0-0.35384615384615387,"[{'xValueOffset': 0, 'xValueDelta': 0.015625, ...",0
1,Env_Time_200_Tr,TrackedEnvelopeTimeSeries,Main bearing,2018-07-08T09:57:18+00:00,1500,135,0.33626923,0.30884615,0.37730769,0.06836538,...,0.81666667,1158,-0.02181111,12.781,0.275436,0.87833333,12.3,WPS-ActivePower 0.0-0.35384615384615387,"[{'xValueOffset': 0, 'xValueDelta': 0.015625, ...",0
2,Env_Time_200_Tr,TrackedEnvelopeTimeSeries,Main bearing,2018-07-08T03:51:47+00:00,1500,131,0.3675,0.32307692,0.39692308,0.07440769,...,0.76666667,1195.6,-0.02209444,13.202,0.28626,0.89413889,12.3,WPS-ActivePower 0.35423076923076924-0.44230769...,"[{'xValueOffset': 0, 'xValueDelta': 0.015625, ...",0
3,Env_Time_200_Tr,TrackedEnvelopeTimeSeries,Main bearing,2018-07-07T21:41:18+00:00,1500,100,0.58065385,0.56576923,0.59807692,0.02068769,...,0.83333333,1567.7,0.05044444,17.31,0.33196,0.91847222,12.3,WPS-ActivePower 0.54-0.6369230769230769,"[{'xValueOffset': 0, 'xValueDelta': 0.015625, ...",1
4,Env_Time_200_Tr,TrackedEnvelopeTimeSeries,Main bearing,2018-07-07T15:37:56+00:00,1500,122,0.46826923,0.40384615,0.54692308,0.11609231,...,0.81708333,1284.8,-0.02214889,14.183,0.312228,0.88052778,12.3,WPS-ActivePower 0.4426923076923077-0.539615384...,"[{'xValueOffset': 0, 'xValueDelta': 0.015625, ...",1


In [114]:
# names = ['a.a','b','c']
# v = [['2018-07-08T22:08:15+00:00','1','2'],
#      ['2018-07-06T09:57:18+00:00','11','22'],['2018-07-18T03:51:47+00:00','111','222']]
# import numpy as np
# import pandas as pd
# npv = np.array(v)
# df = pd.DataFrame(npv,columns=names)
# add_v = [0,0,0]
# df['add'] = add_v
# df['life'] = df['a.a'].apply(lambda x:cal_left_life('2018-07-08T22:08:15+00:00',x))
# df

,a.a,b,c,add,life
0,2018-07-08T22:08:15+00:00,1,2,0,0
1,2018-07-06T09:57:18+00:00,11,22,0,2
2,2018-07-18T03:51:47+00:00,111,222,0,-10
